In [1]:
import math
#import random
#import re
from functools import reduce
from itertools import permutations
# import pandas as pd
#import sympy
#sympy.init_printing()

import sys; sys.path.append("../modules")
from Permutation import *

from tqdm.notebook import tqdm
from multiprocessing import Pool, cpu_count

# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

from knot_floer_homology import *
#help(pd_to_hfk)

## Creating PD-codes for knots with given crossing number

In [ ]:
N = 20
perms = permutations(range(N+1,2*N+1), N)
print(len(list(perms)))

In [2]:
N = 6

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

def calc_hfk(perm):
    if (perm[0] == 2*N) or (perm[N-1] == N+1): return None
    hfk_data = []
    for signs in range(4**(N-1)): # dividing by 4 corresponds to giving the quatiant of the set of all signs w.r.t. reverting all signs at the same time
        pd = [cross(i+1, perm[i], ((signs >> 2*i) & 0b11)) for i in range(N)]
        cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
        pd_perm = reduce((lambda x,y: x*y), cs) #; print(f"{pd_perm}\n\n")
        num_of_orbits = len(cycle_decomp(pd_perm))
        if num_of_orbits - N == 2:
            hfk = pd_to_hfk(pd)
            if hfk['fibered'] and hfk['seifert_genus'] > 0:
                hfk_data.append({'pd': pd, 'hfk': hfk})
    return hfk_data

hfk_data = []
fct = math.factorial(N)
with Pool(processes=8) as pool:
    output = list(pool.imap(calc_hfk, permutations(range(N+1,2*N+1), N)))
#    output = list(tqdm(pool.imap(calc_hfk, permutations(range(N+1,2*N+1), N)),
#                       total=fct, desc='hfk calc'))
    for v in output:
        if v:
            hfk_data += v

hfk_data_slim = [hfk_data[0]]
for data in hfk_data:
    not_dup = True
    for other in hfk_data_slim:
        if data['hfk'] == other['hfk']:
            not_dup = False
            #print(f"{data}\n{other}\n")
    if not_dup:
        hfk_data_slim.append(data)
        
for data in hfk_data_slim:
#    print(f"{data['pd']}\n hfk: {data['hfk']['ranks']}, {data['hfk']['seifert_genus']}\n")
    print(f"{data['pd']}\n hfk: {data['hfk']}\n")    

[(1, 8, 2, 9), (7, 2, 8, 3), (3, 11, 4, 10), (11, 5, 12, 4), (12, 5, 1, 6), (9, 7, 10, 6)]
 hfk: {'L_space_knot': False, 'epsilon': 0, 'fibered': True, 'modulus': 2, 'nu': 0, 'ranks': {(-1, -1): 1, (0, 0): 3, (1, 1): 1}, 'seifert_genus': 1, 'tau': 0, 'total_rank': 5}

[(8, 2, 9, 1), (2, 8, 3, 7), (3, 11, 4, 10), (11, 5, 12, 4), (12, 5, 1, 6), (9, 7, 10, 6)]
 hfk: {'L_space_knot': True, 'epsilon': 1, 'fibered': True, 'modulus': 2, 'nu': 1, 'ranks': {(-1, -2): 1, (0, -1): 1, (1, 0): 1}, 'seifert_genus': 1, 'tau': 1, 'total_rank': 3}

[(8, 2, 9, 1), (7, 2, 8, 3), (10, 3, 11, 4), (4, 11, 5, 12), (12, 5, 1, 6), (9, 7, 10, 6)]
 hfk: {'L_space_knot': True, 'epsilon': -1, 'fibered': True, 'modulus': 2, 'nu': 0, 'ranks': {(-1, 0): 1, (0, 1): 1, (1, 2): 1}, 'seifert_genus': 1, 'tau': -1, 'total_rank': 3}

[(1, 8, 2, 9), (7, 2, 8, 3), (3, 11, 4, 10), (11, 5, 12, 4), (5, 1, 6, 12), (9, 7, 10, 6)]
 hfk: {'L_space_knot': False, 'epsilon': 1, 'fibered': True, 'modulus': 2, 'nu': 1, 'ranks': {(-2, -3)

In [ ]:
N = 17

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

def calc_hfk(signs):
    for perm in permutations(range(N+1,2*N+1), N):
        if (perm[0] == 2*N) or (perm[N-1] == N+1): return None
        pd = [cross(i+1, perm[i], ((signs >> 2*i) & 0b11)) for i in range(N)]
        #---
        cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
        pd_perm = reduce((lambda x,y: x*y), cs) #; print(f"{pd_perm}\n\n")
        num_of_orbits = len(cycle_decomp(pd_perm))
        pd_data = []
        if num_of_orbits - N == 2:
            hfk = pd_to_hfk(pd)
            if hfk['fibered'] and hfk['seifert_genus'] > 0:
                sg = hfk['seifert_genus']
                ranks = hfk['ranks']
                for pg in ranks.keys():
                    if pg[0] == sg:
                        lmd = pg[1]
                if lmd < 0:
                    print(pd)
                    return pd
        return None

with Pool(processes=8) as pool:
    list(tqdm(pool.imap(calc_hfk, range(4**(N-1))), total = 4**(N-1)))
#print(len(output), math.factorial(N)*(4**(N-1)))
#print(output[:10])

In [ ]:
N = 6
hfk_data = []

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

#for perm in tqdm(permutations(range(N+1,2*N+1), N), total=math.factorial(N)): # iteratation on permutations on {N+1, ... ,2*N+1} 
for perm in permutations(range(N+1,2*N+1), N): # iteratation on permutations on {N+1, ... ,2*N+1} 
    if (perm[0] != 2*N) and (perm[N-1] != N+1):
        for signs in range(4**(N-1)): # dividing by 4 corresponds to giving the quatiant of the set of all signs w.r.t. reverting all signs at the same time
            pd = [cross(i+1, perm[i], ((signs >> 2*i) & 0b11)) for i in range(N)]
            cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
            pd_perm = reduce((lambda x,y: x*y), cs)
            num_of_orbits = len(cycle_decomp(pd_perm))
            if num_of_orbits - N == 2:
                hfk = pd_to_hfk(pd)
                if hfk['fibered'] and hfk['seifert_genus'] > 0:
                    hfk_data.append({'pd': pd, 'hfk': hfk})
                    
hfk_data_slim = [hfk_data[0]]
for data in hfk_data:
    not_dup = True
    for other in hfk_data_slim:
        if data['hfk'] == other['hfk']:
            not_dup = False
            #print(f"{data}\n{other}\n")
    if not_dup:
        hfk_data_slim.append(data)
        
for data in hfk_data_slim:
#    print(f"{data['pd']}\n hfk: {data['hfk']['ranks']}, {data['hfk']['seifert_genus']}\n")
    print(f"{data['pd']}\n hfk: {data['hfk']}\n")

In [ ]:
def list_permutations(elements):
    if len(elements) <= 1:
        yield elements
    else:
        for p in list_permutations(elements[1:]):
            for i in range(len(elements)):
                yield p[:i] + elements[0:1] + p[i:]

# Accept input from the user
# nums = [1,2,3,4]
N = 10
nums = list(range(N+1,2*N+1))
print("Original list of elements:",nums)

# Generate and print all permutations
count = 0
for p in tqdm(list_permutations(nums), total=math.factorial(N)):
    count += 1
print(count)

In [ ]:
%%time

N = 10
nums = list(range(N+1,2*N+1))

# Generate and print all permutations
count = 0
for p in tqdm(permutations(nums, N), total=math.factorial(N)):
    count += 1
print(count)

In [ ]:
%%time

N = 10
nums = list(range(N+1,2*N+1))

# Generate and print all permutations
count = 0
for p in permutations(nums, N):
    count += 1
print(count)

In [ ]:
N = 10
hfk_data = []

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

def calc_hfk(perm):
    if (perm[0] == 2*N) and (perm[N-1] == N+1):
        return None
    else:
        for signs in range(4**(N-1)):
            data = []
            pd = [cross(i+1, perm[i], (signs >> 2*i) & 0b11) for i in range(N)]
            if pd == [(5,2,6,1), (6,3,1,2), (4,4,5,3)]:
                print(f"{perm} --> {perm[N-1], N+1}, {pd}, \n")
                cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
                pd_perm = reduce((lambda x,y: x*y), cs)
                num_of_orbits = len(cycle_decomp(pd_perm))
                if num_of_orbits - N == 2:
                    hfk = pd_to_hfk(pd)
                    if hfk['fibered'] and hfk['seifert_genus'] > 0:
                        data.append({'pd': pd, 'hfk': hfk})
        return data
    
for perm in tqdm(permutations(range(N+1,2*N+1), N), total=math.factorial(N)):
    data = calc_hfk(perm)
    if data is not None:
        hfk_data += data
                    
hfk_data_slim = [hfk_data[0]]
for data in hfk_data:
    not_dup = True
    for other in hfk_data_slim:
        if data['hfk'] == other['hfk']:
            not_dup = False
            #print(f"{data}\n{other}\n")
    if not_dup:
        hfk_data_slim.append(data)
        
for data in hfk_data_slim:
#    print(f"{data['pd']}\n hfk: {data['hfk']['ranks']}, {data['hfk']['seifert_genus']}\n")
    print(f"{data['pd']}\n hfk: {data['hfk']}\n")

## scratch

In [ ]:
pd = [(5,2,6,1), (6,3,1,2), (4,4,5,3)]

cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
pd_perm = reduce((lambda x,y: x*y), cs)

In [ ]:
hfk = pd_to_hfk([[1,5,2,4],[5,3,6,2],[3,1,4,6]])
print(f"{hfk}")

In [ ]:
hfk = pd_to_hfk([[1,4,2,5],[3,1,4,6],[5,3,6,2]])
print(f"{hfk}")

In [ ]:
hfk = pd_to_hfk([[1,5,2,4],[5,2,6,3],[3,1,4,6]])
print(f"{hfk}")

In [ ]:
[bin(x) for x in range(2**3)]

In [ ]:
k= 6
for i in range(3):
    flag = bin(k & 2**i)
    print(flag)

In [ ]:
print(cs[0], cs[0].image)

print(cs[1]*cs[0], (cs[1]*cs[0]).image)